# LAB EXTRA 2
### Hecho por: Joshua Sancho y Steven Solís

## Importar Librerías

In [2]:
import os
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

## Procesamiento de Dataset

In [3]:
def resize_images_and_save(dataset_path, save_resized_path):
    # Create the directory if it doesn't exist
    if not os.path.exists(save_resized_path):
        os.makedirs(save_resized_path)
    
    # Loop through each class folder
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        
        # Create a directory for the resized images of this class
        class_save_path = os.path.join(save_resized_path, class_name)
        if not os.path.exists(class_save_path):
            os.makedirs(class_save_path)
        
        # Loop through each image file in the class folder
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            
            # Read and convert the image to grayscale
            image = cv2.imread(image_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Resize the grayscale image to 40x40
            resized_image = cv2.resize(gray_image, (40, 40))
            
            # Save the resized image
            save_image_path = os.path.join(class_save_path, image_file)
            cv2.imwrite(save_image_path, resized_image)

In [ ]:
resize_images_and_save(r'dataset/train', r'dataset/train_resized')
resize_images_and_save(r'dataset/valid',  r'dataset/val_resized')
resize_images_and_save(r'dataset/test',  r'dataset/test_resized')

In [ ]:
def create_dataframe_from_images(dataset_path):
    data = []
    
    # Loop through each class folder
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        
        # Loop through each image file in the class folder
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            
            # Append the image path and class name to the data list
            data.append({'image_path': image_path, 'class': class_name})
    
    # Create a pandas DataFrame from the data list
    df = pd.DataFrame(data)
    return df

In [ ]:
train = create_dataframe_from_images(r'../LAB_Extra_2/dataset/train_resized')
test = create_dataframe_from_images(r'../LAB_Extra_2/dataset/test_resized')
val = create_dataframe_from_images(r'../LAB_Extra_2/dataset/val_resized')

In [ ]:
# Define data augmentation and normalization parameters for training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,      # Rotate images randomly up to 40 degrees
    width_shift_range=0.2,  # Shift images horizontally up to 20% of the width
    height_shift_range=0.2, # Shift images vertically up to 20% of the height
    shear_range=0.2,        # Shear intensity
    zoom_range=0.2,         # Zoom in randomly up to 20%
    horizontal_flip=True,   # Flip images horizontally
    fill_mode='nearest'     # Fill in newly created pixels after rotation or shifting
)

# Create a data generator for training data with one-hot encoded labels
train_generator = train_datagen.flow_from_directory(
    '../LAB_Extra_2/dataset/train',
    target_size=(40, 40),
    batch_size=5,
    class_mode='binary',  # Change class_mode to 'categorical'
    shuffle=True)  # Shuffle the data

# Define data augmentation and normalization parameters for validation data
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create a data generator for validation data with one-hot encoded labels
val_generator = val_datagen.flow_from_directory(
    '../LAB_Extra_2/dataset/valid',
    target_size=(40, 40),
    batch_size=5,
    class_mode='binary',  # Change class_mode to 'categorical'
    shuffle=False)  # No need to shuffle validation data

## Arquitectura de Red Neuronal Convolucional

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(40, 40, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),

    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Entrenamiento

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Define model checkpoint callback to save the best model
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Compile the model with a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',  # Change the loss function
              metrics=['accuracy'])

# Train the model for 100 epochs
history = model.fit(train_generator, validation_data=val_generator, epochs=100, callbacks=[model_checkpoint])

In [ ]:
# Get the training metrics
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']

# Get the validation metrics
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Plot training and validation loss
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Conclusión
Se logró mejorar la precisión del modelo a un 99% con una pérdida del 4% en valores de validación!